原始图像来源于隋安的FFD项目，图像生成代码修改自隋安的FFD项目

In [1]:
import numpy as np
import SimpleITK as sitk
from PIL import Image
from matplotlib import pyplot as plt

from func import *

In [2]:
# 生成测试数据
torch.manual_seed(926)  # random seed
image_path = './data/raw_data/Case1032_stacked_CINE_segmented_LAX_4Ch_20180801_1.nii.gz'  # 图像路径
itk_img = sitk.ReadImage(image_path)
img = sitk.GetArrayFromImage(itk_img)
chosen_slice = img[0, :, :]/1.0  # 取一个切片：3D → 2D  (174, 192)
h = 40  # 网格间距;控制点数量与此有关,间距越小,控制点越多,变形能力越强
sigma = 10  # 形变程度,值为0时只作仿射变换
target_img = torch.tensor(chosen_slice)
target_img_tensor = target_img.float().unsqueeze(0)  # 在位置0处增加一个维度
mesh_size = (torch.tensor(target_img.shape) / h).floor().int() + 4  # 由节点构成的网格大小: 8 × 8 (h = 40)
# mesh_displacement = torch.randn((2, mesh_size[0], mesh_size[1])) * sigma  # 随机生成FFD网格在两个维度上各自的位移, shape: 2 × 8 × 8
mesh_displacement=torch.load('./data/FFD/parameter/mesh_displacement.pt')   # 为确保复现，从文件中读取之前所用的位移
img_coord = make_coord(target_img_tensor.shape[1:])  # 2 x H x W: 2 × 174 × 192

target_coord = FFD_transformation(img_coord, mesh_displacement, h)  # FFD自由形变, 参数: 位移, h
interpolated_img_tensor = linear_interpolation(target_coord, target_img_tensor)  # 利用插值获取新坐标处的像素值

grid_img = torch.ones_like(target_img_tensor)
grid_img[:, :, ::20] = 0
grid_img[:, ::20, :] = 0  # 以上三行用于制作网格状图像, 用以可视化deformation field
deformation_field = linear_interpolation(target_coord, grid_img)
reference_img = Image.fromarray(np.uint8(normalize(chosen_slice)*255))
reference_img.save('./data/FFD/image/test_reference_img.jpg')
floating_img = Image.fromarray(np.uint8(normalize(interpolated_img_tensor.squeeze(0).numpy())*255))

floating_img.save('./data/FFD/image/test_floating_img.jpg')  # sigma > 0时使用
deformation_field_img = Image.fromarray(np.uint8(normalize(deformation_field.squeeze(0).numpy())*255))
deformation_field_img.save('./data/FFD/image/test_deformation_field.jpg')
different_img = Image.fromarray(np.uint8(normalize((interpolated_img_tensor-target_img_tensor).detach().squeeze(0).numpy())*255))
different_img.save('./data/FFD/image/test_different_img.jpg')

/home/gonghaiyu/miniconda3/envs/pytorch/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
torch.save(mesh_displacement, './data/FFD/parameter/mesh_displacement.pt')  # 保存位移参数，以便复现

In [7]:
# 生成训练数据，使用生成测试数据所用的形变场
image_path = './data/raw_data/Case1032_stacked_CINE_segmented_SAX_InlineVF_20180801_3.nii.gz'  # 图像路径
itk_img = sitk.ReadImage(image_path)
img = np.transpose(sitk.GetArrayFromImage(itk_img),(0,1,3,2))

for idx in range(10):
    chosen_slice = img[0,idx, :, :]/1.0  # 取一个切片：3D → 2D  (174, 192)

    target_img = torch.tensor(chosen_slice)
    target_img_tensor = target_img.float().unsqueeze(0)  # 在位置0处增加一个维度

    interpolated_img_tensor = linear_interpolation(target_coord, target_img_tensor)  # 利用插值获取新坐标处的像素值

    deformation_field = linear_interpolation(target_coord, grid_img)
    reference_img = Image.fromarray(np.uint8(normalize(chosen_slice)*255))
    reference_img.save(f'./data/FFD/image/train_reference_img_{idx}.jpg')
    floating_img = Image.fromarray(np.uint8(normalize(interpolated_img_tensor.squeeze(0).numpy())*255))
    # floating_img.save('image/generate_image/floating_img_affine_only.jpg')  # sigma = 0时使用
    floating_img.save(f'./data/FFD/image/train_floating_img_{idx}.jpg')  # sigma > 0时使用
    deformation_field_img = Image.fromarray(np.uint8(normalize(deformation_field.squeeze(0).numpy())*255))
    deformation_field_img.save(f'./data/FFD/image/train_deformation_field_{idx}.jpg')
    different_img = Image.fromarray(np.uint8(normalize((interpolated_img_tensor-target_img_tensor).detach().squeeze(0).numpy())*255))
    different_img.save(f'./data/FFD/image/train_different_img_{idx}.jpg')